In [22]:
from pymongo import MongoClient

In [23]:
client = MongoClient("mongodb://mongodb:27017/")

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("myApp") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.2') \
.getOrCreate()

In [25]:
mers = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://mongodb:27017/epidemics.mers").load()

In [5]:
#days_week = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
#"mongodb://mongodb:27017/epidemics1.days_week").load()

In [27]:
mers.createOrReplaceTempView("mers")
#days_week.createOrReplaceTempView("days_week")

In [43]:
mers = spark.sql("SELECT _id, region AS country, year, week, int(new_cases) AS cases, \
COALESCE(int((sum(new_cases) over (partition by region ORDER BY year, int(week) asc))), 0) as cumul_cases FROM mers")

In [44]:
mers.show()

+--------------------+-----------------+----+----+-----+-----------+
|                 _id|          country|year|week|cases|cumul_cases|
+--------------------+-----------------+----+----+-----+-----------+
|[5eaad581681795c9...|Republic of Korea|2012|  12|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  13|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  14|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  15|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  16|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  17|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  18|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  19|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  20|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  21|    0|          0|
|[5eaad581681795c9...|Republic of Korea|2012|  22|    0|          0|
|[5eaad581681795c9...|Republic of 

In [15]:
#mers = spark.sql("SELECT mers._id, region, mers.year, mers.week, week_start_date, week_end_date, int(new_cases) FROM mers left join days_week on \
#(mers.year=days_week.year) AND (mers.week=days_week.week)")

In [22]:
#mers.createOrReplaceTempView("mers")

In [31]:
#spark.sql("SELECT * FROM mers ORDER BY week_start_date").show()

In [11]:
#mers.show()

In [45]:
mers.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics2.epidemics_mers").save()